In [1]:
library(readr)
library(leaflet)
library(crosstalk)
library(sp)
library(dplyr)

Warning message:
“replacing previous import by ‘shiny::includeHTML’ when loading ‘crosstalk’”Warning message:
“replacing previous import by ‘shiny::knit_print.shiny.tag’ when loading ‘crosstalk’”Warning message:
“replacing previous import by ‘shiny::code’ when loading ‘crosstalk’”Warning message:
“replacing previous import by ‘shiny::includeScript’ when loading ‘crosstalk’”Warning message:
“replacing previous import by ‘shiny::includeMarkdown’ when loading ‘crosstalk’”Warning message:
“replacing previous import by ‘shiny::tags’ when loading ‘crosstalk’”Warning message:
“replacing previous import by ‘shiny::is.singleton’ when loading ‘crosstalk’”Warning message:
“replacing previous import by ‘shiny::withTags’ when loading ‘crosstalk’”Warning message:
“replacing previous import by ‘shiny::img’ when loading ‘crosstalk’”Warning message:
“replacing previous import by ‘shiny::tagAppendAttributes’ when loading ‘crosstalk’”Warning message:
“replacing previous import by ‘shiny::knit_print.shiny

In [2]:
journeys_text <- '"id","from","to","duration_car","duration_car_traffic","distance","duration","speedup_traffic","speedup","lat_from","long_from","lat_to","long_to"
1,"St Pancras","Birmingham",139.733333333333,155.7,203.495,105,1.48285714285714,1.33079365079365,51.5318810626864,-0.12558183674287,52.4774461110237,-1.89876494598884
2,"St Pancras","Leeds",213.116666666667,216.283333333333,310.505,149,1.45156599552572,1.43031319910515,51.5318810626864,-0.12558183674287,53.7954036883147,-1.54648959940816
3,"St Pancras","Glasgow",405.916666666667,416.9,662.744,291,1.43264604810997,1.39490263459336,51.5318810626864,-0.12558183674287,55.8598157580745,-4.25635960252115
4,"St Pancras","Sheffield",186.233333333333,189.733333333333,265.883,137,1.38491484184915,1.35936739659367,51.5318810626864,-0.12558183674287,53.3779466312297,-1.46058230158514
5,"St Pancras","Bradford",226.083333333333,229.216666666667,325.379,189,1.21278659611993,1.19620811287478,51.5318810626864,-0.12558183674287,53.7908548393771,-1.74808433128547
6,"St Pancras","Edinburgh",434.1,437.916666666667,645.885,319,1.37277951933124,1.36081504702194,51.5318810626864,-0.12558183674287,55.9524496827932,-3.18680284123936
7,"St Pancras","Liverpool Central",236.066666666667,246.866666666667,352.76,198,1.24680134680135,1.19225589225589,51.5318810626864,-0.12558183674287,53.404351231227,-2.97782044521494
8,"St Pancras","Manchester",229.35,238.783333333333,335.821,154,1.55054112554112,1.48928571428571,51.5318810626864,-0.12558183674287,53.4771110532056,-2.22946942556054
9,"St Pancras","Bristol",148.116666666667,152.233333333333,193.611,143,1.06456876456876,1.03578088578089,51.5318810626864,-0.12558183674287,51.4486612575213,-2.57999158987809'

journeys <- read.csv(textConnection(journeys_text))

In [3]:
colors <- colorBin("PuOr", bins=c(0, 0.5, 0.8, 0.9, 1.1, 1.2, 1.5, 3))

p <- leaflet() %>%
addProviderTiles("CartoDB.DarkMatter")

for(j in 1:nrow(journeys)) {
    p <- p %>% addPolylines(lat=c(journeys$lat_from[j], journeys$lat_to[j]),
                            lng=c(journeys$long_from[j], journeys$long_to[j]),
                           color=colors(journeys$speedup[j]), weight=1,
                           popup = paste(journeys$from[j], "-", journeys$to[j], "|", round(journeys$speedup[j], 1)),
                           group=journeys$from[j],
                           opacity=0.5)
}

tf = 'myfigure.html'
htmlwidgets::saveWidget(p, file = tf, selfcontained = F)
IRdisplay::display_html(paste("<iframe height='600' width='1000' src=' ", tf, " ' ","/>"))

In [5]:
# Make a SpatialLines data frame to store all journeys
a <- SpatialLines(lapply(journeys$id, function(x){
    
    Lines(Line(cbind(c(filter(journeys, id==x)$long_from, filter(journeys, id==x)$long_to),
                    c(filter(journeys, id==x)$lat_from, filter(journeys, id==x)$lat_to))), ID = x)
}))
b <- SpatialLinesDataFrame(a, data=journeys)

In [6]:
# Wrap data in SharedData
journeys_sd <- SharedData$new(b)

# Create filter for journeys
bscols(filter_slider("speedup", "Speedup", journeys_sd, column = ~speedup),

leaflet(journeys_sd) %>%
addProviderTiles("CartoDB.DarkMatter") %>%
addPolylines(popup = ~from))

ERROR: Error: Do not know how to find data associated with `x`
